In [42]:
import pandas as pd
import datetime 
import numpy as np
import pickle
import matplotlib.pyplot as plt
import dtale
import re
import seaborn as sns
from functools import reduce

%matplotlib inline

In [2]:
truck = pd.read_csv('../data/average_truck_speeds_on_us_highways_2009.csv', skiprows=2)
#source: US Dept. of Transportation - https://ops.fhwa.dot.gov/freight/freight_analysis/nat_freight_stats/docs/10factsfigures/table3_8.htm

rail = pd.read_csv('../data/railroad_speed_data.csv')
#source: US Surface Transportation Board - https://www.stb.gov/reports-data/rail-service-data/

In [3]:
#truck.head()
#truck.info()
truck = truck[['Interstate Route', 'Average Operating Speed','Peak Period Average  Speed ','Non-Peak Period Average Speed']]


In [4]:
truck.head()
truck = truck.rename(columns ={
    'Interstate Route':'route',
    'Average Operating Speed':'avg_speed',
    'Peak Period Average  Speed ':'peak_avg_speed',
    'Non-Peak Period Average Speed':'non_peak_avg_speed'
})

In [5]:
truck.head()

,route,avg_speed,peak_avg_speed,non_peak_avg_speed
0,5,52.8,52.0,53.1
1,10,57.4,56.7,57.6
2,15,56.7,56.2,56.9
3,20,59.2,58.8,59.3
4,24,57.2,56.6,57.4


In [6]:
truck['avg_speed'].mean()

56.284

In [7]:
#truckspeed = truck['avg_speed'].mean().rename_axis('truck').reset_index(name='avg_mph_speed')

truckspeed = pd.DataFrame({
    "type" : ["truck"],
    "avg_mph_speed" : [56.284]
})

In [8]:
rail.head()

,Railroad/\nRegion,Category No.,Sub-Category,Measure,Variable,Sub-Variable,3/29/2017,4/5/2017,4/12/2017,4/19/2017,...,9/29/2021,10/6/2021,10/13/2021,10/20/2021,10/27/2021,11/3/2021,11/10/2021,11/17/2021,11/24/2021,12/1/2021
0,BNSF,1.0,NaN,Average Train Speed (MPH),Intermodal,NaN,32.7,31.6,32.3,32.5,...,33.0,33.2,33.5,33.2,33.0,32.4,31.8,32.3,32.0,33.8
1,BNSF,1.0,NaN,Average Train Speed (MPH),Grain unit,NaN,20.5,21.8,22.0,22.3,...,25.2,23.5,23.7,23.4,22.9,23.1,23.3,23.8,23.0,24.2
2,BNSF,1.0,NaN,Average Train Speed (MPH),Coal unit,NaN,24.1,24.1,23.6,23.4,...,22.9,21.7,22.4,22.3,22.1,21.9,22.6,22.3,22.6,24.1
3,BNSF,1.0,NaN,Average Train Speed (MPH),Automotive unit,NaN,24.9,24.7,24.8,26.5,...,25.1,26.7,26.6,27.2,26.3,25.9,25.1,26.0,24.8,26.3
4,BNSF,1.0,NaN,Average Train Speed (MPH),Crude oil unit,NaN,22.2,22.4,23.5,22.6,...,23.6,23.7,22.5,23.1,22.8,22.1,21.8,22.7,21.6,22.2


In [9]:
rail['Measure'].value_counts()

Grain Car Order Fulfillment Metrics (Count)                                                                  1192
Weekly total grain cars loaded and billed, reported by State and Type of Car Ordering System                  885
Weekly Carloads By 22 Commodity Categories                                                                    462
Weekly Average Number of Trains Holding Per Day by Train Type and Cause                                       220
Weekly Average Number of Loaded and Empty Cars in Revenue Service That Have Not Moved 48 Hours Or Greater     126
Average Terminal Dwell Time (Excluding Cars on Run Through Trains) (Hours)                                     78
Coal Unit Train Loadings or Carloadings by Coal Production Region (Count)                                      76
Cars On Line (Count)                                                                                           63
Average Train Speed  (MPH)                                                              

In [10]:
railspeed = rail.loc[rail.Measure =='Average Train Speed  (MPH)']

In [11]:
railspeed.head()
railspeed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 2736
Columns: 251 entries, Railroad/
Region to 12/1/2021
dtypes: float64(1), object(250)
memory usage: 110.2+ KB


In [15]:
railspeed['Variable'].value_counts()

#note: my interest is primarily in food transport systems.  And there is a specific "grain unit" type of train data
#but rail is used to transport much more than just grain.  
#my products are in multiple categories

Manifest           7
Intermodal         7
Ethanol unit       6
Crude oil unit     6
Grain unit         6
Automotive unit    6
Coal unit          6
System             6
Other Unit         1
Coal               1
Crude              1
Grain              1
System (U.S.)      1
Ethanol            1
Name: Variable, dtype: int64

In [19]:
railspeed2 = railspeed.loc[railspeed['Variable'].isin(['Manifest','Intermodal','Grain unit','System','Other Unit','Grain','System (U.S.)'])]

In [22]:
railspeed2['Variable'].value_counts()
railspeed2.head()


,Railroad/\nRegion,Category No.,Sub-Category,Measure,Variable,Sub-Variable,3/29/2017,4/5/2017,4/12/2017,4/19/2017,...,9/29/2021,10/6/2021,10/13/2021,10/20/2021,10/27/2021,11/3/2021,11/10/2021,11/17/2021,11/24/2021,12/1/2021
0,BNSF,1.0,NaN,Average Train Speed (MPH),Intermodal,NaN,32.7,31.6,32.3,32.5,...,33.0,33.2,33.5,33.2,33.0,32.4,31.8,32.3,32.0,33.8
1,BNSF,1.0,NaN,Average Train Speed (MPH),Grain unit,NaN,20.5,21.8,22.0,22.3,...,25.2,23.5,23.7,23.4,22.9,23.1,23.3,23.8,23.0,24.2
6,BNSF,1.0,NaN,Average Train Speed (MPH),Manifest,NaN,21.2,21.4,21.4,21.4,...,21.9,21.9,21.7,21.9,22.1,20.7,21.6,21.4,21.1,21.9
7,BNSF,1.0,NaN,Average Train Speed (MPH),System,NaN,24.8,25.0,25.0,25.3,...,25.7,25.2,25.4,25.3,25.1,24.6,24.8,24.9,24.5,26.0
524,CN,1.0,NaN,Average Train Speed (MPH),Grain,NaN,25.0,24.1,25.5,24.6,...,19.5,24.9,23.1,24.0,24.8,24.8,23.3,25.4,25.4,25.4


In [26]:
railspeed2 = railspeed2.drop(['Railroad/\nRegion','Category No.','Sub-Category','Measure','Variable','Sub-Variable'], axis = 1)

In [31]:
railspeed2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 2736
Columns: 246 entries, 3/29/2017 to avg_speed
dtypes: float64(1), object(245)
memory usage: 57.0+ KB


In [33]:
railspeed2 = railspeed2.apply(pd.to_numeric)

In [34]:
railspeed2['avg_speed'] = railspeed2.mean(axis=1)
railspeed2

,3/29/2017,4/5/2017,4/12/2017,4/19/2017,4/26/2017,5/3/2017,5/10/2017,5/17/2017,5/24/2017,5/31/2017,...,10/6/2021,10/13/2021,10/20/2021,10/27/2021,11/3/2021,11/10/2021,11/17/2021,11/24/2021,12/1/2021,avg_speed
0,32.7,31.6,32.3,32.5,33.4,33.3,33.0,32.8,31.8,32.5,...,33.2,33.5,33.2,33.0,32.4,31.8,32.3,32.0,33.8,33.028163
1,20.5,21.8,22.0,22.3,22.5,22.8,22.1,22.4,21.5,22.1,...,23.5,23.7,23.4,22.9,23.1,23.3,23.8,23.0,24.2,23.097143
6,21.2,21.4,21.4,21.4,21.5,21.6,20.7,20.5,20.6,20.6,...,21.9,21.7,21.9,22.1,20.7,21.6,21.4,21.1,21.9,21.563265
7,24.8,25.0,25.0,25.3,25.7,25.7,25.1,25.0,24.5,24.8,...,25.2,25.4,25.3,25.1,24.6,24.8,24.9,24.5,26.0,25.206531
524,25.0,24.1,25.5,24.6,24.3,24.4,25.8,24.6,25.5,26.1,...,24.9,23.1,24.0,24.8,24.8,23.3,25.4,25.4,25.4,23.533061
525,28.4,27.9,27.1,27.1,27.1,27.4,26.4,28.0,27.7,27.5,...,28.7,27.9,26.8,27.8,27.0,27.6,26.8,28.4,28.4,26.189796
526,22.9,22.9,22.8,23.2,23.3,23.3,22.3,23.6,23.2,23.4,...,23.4,22.8,23.1,23.2,22.8,23.2,23.0,23.0,23.5,21.908980
527,23.8,23.1,22.8,23.1,23.2,23.0,21.9,23.1,23.0,23.4,...,24.2,23.4,22.6,24.7,23.1,24.1,24.4,25.0,24.9,21.909796
528,24.1,23.9,23.9,24.0,24.2,24.1,23.1,24.5,24.3,24.5,...,24.5,23.9,23.8,24.5,23.9,24.2,24.2,24.4,24.6,22.943673
1045,29.8,29.1,28.5,29.4,30.1,28.7,30.4,30.4,32.8,33.3,...,35.3,36.4,34.5,36.5,34.1,34.9,34.0,34.2,36.7,32.298367


In [35]:
railspeed2['avg_speed'].mean()

24.779812807881775

In [37]:
#out of curiousity - what would it be for all trains?

railspeedtest = railspeed.drop(['Railroad/\nRegion','Category No.','Sub-Category','Measure','Variable','Sub-Variable'], axis = 1)
railspeedtest = railspeedtest.apply(pd.to_numeric)
railspeedtest['avg_speed'] = railspeedtest.mean(axis=1)
railspeedtest['avg_speed'].mean()

#guess it's not much surprise that when you factor in things like oil cars the speed goes down.  
#they probably shouldn't travel as fast as regular cargo can

22.744553218785075

In [39]:
railspeed3 = pd.DataFrame({
    "type" : ["rail"],
    "avg_mph_speed" : [24.779812807881775]
})

In [12]:
#I want the avg mph.  for knots, the conversion is 1.15

boat_plane_speed = pd.DataFrame({
    "type" : ["ship_slow", "ship_avg", "cargo_plane"],
    "avg_mph_speed" : ["22","26","560"]
})

#source: boats: https://transportgeography.org/contents/chapter4/transportation-and-energy/fuel-consumption-containerships/
#https://www.marineinsight.com/guidelines/speed-of-a-ship-at-sea/
#source: plane: https://www.flightdeckfriend.com/ask-a-pilot/how-fast-do-commercial-aeroplanes-fly

In [38]:
boat_plane_speed.head()

,type,avg_mph_speed
0,ship_slow,22
1,ship_avg,26
2,cargo_plane,560


In [81]:
dfs = [boat_plane_speed,railspeed3,truckspeed]

speeds = pd.concat(dfs)

In [82]:
#speeds.info()
speeds['avg_mph_speed'] = speeds['avg_mph_speed'].apply(pd.to_numeric)
speeds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 0
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   type           5 non-null      object 
 1   avg_mph_speed  5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 120.0+ bytes


In [83]:
#speeds
speeds['avg_mph_speed'] = speeds['avg_mph_speed'].round(decimals=0)
speeds

,type,avg_mph_speed
0,ship_slow,22.0
1,ship_avg,26.0
2,cargo_plane,560.0
0,rail,25.0
0,truck,56.0
